In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'traffic-signs-data/train.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [2]:
### Replace each question mark with the appropriate value.
import numpy as np

# TODO: Number of training examples
n_train = train['features'].shape[0]

# TODO: Number of testing examples.
n_test = test['features'].shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = train['features'].shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(train['labels']))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


In [3]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.
from sklearn.model_selection import train_test_split

train_features, valid_features, train_labels, valid_labels = train_test_split(
    train['features'],
    train['labels'],
    test_size=0.2,
    random_state=832289)
test_features = test['features']
test_labels = test['labels']

print ("train data size: {0}, validation data size: {1}, test data size: {2}".format(train_features.shape[0],
                                                                                    valid_features.shape[0],
                                                                                    test_features.shape[0]))

train data size: 31367, validation data size: 7842, test data size: 12630


In [4]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten

# Inputs
features = tf.placeholder(tf.float32, (None, 32, 32, 3), name = "features")
label = tf.placeholder(tf.int32, (None), name = "label")
one_hot_label = tf.one_hot(label, n_classes)
    
# Layer 1: Convolutional
weights1 = tf.Variable(tf.truncated_normal([5, 5, 3, 6], mean=0, stddev=0.1), name = 'weights')
biases1 = tf.Variable(tf.zeros(6), name = 'biases')
conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(features, weights1, strides=[1, 1, 1, 1], padding='VALID'), biases1))
conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

# Layer 2: Convolutional
weights2 = tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=0, stddev=0.1), name = 'weights')
biases2 = tf.Variable(tf.zeros(16), name = 'biases')
conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(conv1, weights2, strides=[1, 1, 1, 1], padding='VALID'), biases2))
conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

#keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    
# Layer 3: Full Connected
flat_dim = 5 * 5 * 16
fc0 = flatten(conv2)
weights3 = tf.Variable(tf.truncated_normal([flat_dim, 120], mean=0, stddev=0.1), name = 'weights')
biases3 = tf.Variable(tf.zeros(120), name = 'biases')
fc1 = tf.nn.relu(tf.add(tf.matmul(fc0, weights3), biases3))
#fc1 = tf.nn.dropout(fc1, keep_prob)

# Layer 4: Full Connected
weights4 = tf.Variable(tf.truncated_normal([120, 84], mean=0, stddev=0.1), name = 'weights')
biases4 = tf.Variable(tf.zeros(84), name = 'biases')
fc2 = tf.nn.relu(tf.add(tf.matmul(fc1, weights4), biases4))
#fc2 = tf.nn.dropout(fc2, keep_prob)

# Layer 5: Full Connected
weights5 = tf.Variable(tf.truncated_normal([84, n_classes], mean=0, stddev=0.1), name = 'weights')
biases5 = tf.Variable(tf.zeros(n_classes), name = 'biases')
output = tf.add(tf.matmul(fc2, weights5), biases5)

In [5]:
### Train your model here.
### Feel free to use as many code cells as needed.
lrate = 0.001
batch_size = 128
epoches=5

# define the cross entropy loss opertation
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(output, one_hot_label)
loss_operation = tf.reduce_mean(cross_entropy)

# define the Adam optimizer and the training operation
optimizer = tf.train.AdamOptimizer(learning_rate = lrate)
training_operation = optimizer.minimize(loss_operation)

# define the accuracy calculation operation
prediction = tf.argmax(output, 1)
correct_prediction = tf.equal(prediction, tf.argmax(one_hot_label, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
# evaluate the accuracy given a set of images and labels
def evaluate(x, y, step = 0):
    num_examples = len(x)
    total_accuracy = 0
    total_loss = 0.0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        end = offset + batch_size
        batch_x, batch_y = x[offset : end], y[offset : end]
        accuracy, loss = sess.run([accuracy_operation, loss_operation] ,
                                     feed_dict={features: batch_x, label: batch_y})
        total_accuracy += (accuracy * len(batch_x))
        total_loss += (loss * len(batch_x))
    return total_accuracy / num_examples, total_loss / num_examples

In [7]:
from sklearn.utils import shuffle

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    num_examples = len(train_features)
    print("Training...")
    print()
    step = 0
    for i in range(epoches):
        train_featues, train_labels = shuffle(train_features, train_labels)
        for offset in range(0, num_examples, batch_size):
            end = offset + batch_size
            batch_features, batch_labels = train_features[offset : end], train_labels[offset : end]
            sess.run(training_operation, feed_dict={features: batch_features, label: batch_labels})
        train_accuracy, loss = evaluate(train_features, train_labels, step)
        valid_accuracy, _ = evaluate(valid_features, valid_labels, step)
        step += 1
        print("EPOCH {} ...".format(i+1))
        print("Average Loss = {:.3f}".format(loss))
        print("Train Accuracy = {:.3f}".format(train_accuracy))
        print("Valid Accuracy = {:.3f}".format(valid_accuracy))
        print()
        
    # Evaluate on the test datatf.truncated_normal        
    test_accuracy, loss = evaluate(test_features, test_labels)
    print("Test accuracy = {:.3f}".format(test_accuracy))
    

Training...

EPOCH 1 ...
Average Loss = 3.507
Train Accuracy = 0.055
Valid Accuracy = 0.051

EPOCH 2 ...
Average Loss = 3.487
Train Accuracy = 0.058
Valid Accuracy = 0.055

EPOCH 3 ...
Average Loss = 3.487
Train Accuracy = 0.057
Valid Accuracy = 0.056

EPOCH 4 ...
Average Loss = 3.486
Train Accuracy = 0.057
Valid Accuracy = 0.055

EPOCH 5 ...
Average Loss = 3.486
Train Accuracy = 0.057
Valid Accuracy = 0.055

Test accuracy = 0.057
